In [10]:
from DB_mapping import engine
import pandas as pd

cnx = engine.connect()
df = pd.read_sql_table('stacje_pomiarowe', cnx)
df

2023-03-04 16:32:23,565 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-04 16:32:23,570 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("stacje_pomiarowe")
2023-03-04 16:32:23,577 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 16:32:23,585 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("stacje_pomiarowe")
2023-03-04 16:32:23,592 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 16:32:23,630 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2023-03-04 16:32:23,632 INFO sqlalchemy.engine.Engine [raw sql] ('stacje_pomiarowe',)
2023-03-04 16:32:23,636 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("stacje_pomiarowe")
2023-03-04 16:32:23,639 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-04 16:32:23,643 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("stacje_pomiarowe")
2023-03-04 16:32:23,645 INFO sqlalchemy.engine.Eng

,station,stationName,gegrLat,gegrLon,loc_id,addressStreet,city_name,communeName,districtName,provinceName
0,11,Czerniawa,50,15,142,ul. Strażacka 7,Czerniawa,Świeradów-Zdrój,lubański,DOLNOŚLĄSKIE
1,16,"Dzierżoniów, ul. Piłsudskiego",50,16,198,ul. Piłsudskiego 26,Dzierżoniów,Dzierżoniów,dzierżoniowski,DOLNOŚLĄSKIE
2,38,"Kłodzko, ul. Szkolna",50,16,368,ul. Szkolna 8,Kłodzko,Kłodzko,kłodzki,DOLNOŚLĄSKIE
3,52,"Legnica, al. Rzeczypospolitej",51,16,453,al. Rzeczypospolitej 10/12,Legnica,Legnica,Legnica,DOLNOŚLĄSKIE
4,70,"Oława, ul. Żołnierzy Armii Krajowej",50,17,642,ul. Żołnierzy AK 9,Oława,Oława,oławski,DOLNOŚLĄSKIE
...,...,...,...,...,...,...,...,...,...,...
203,17160,"Pułtusk, ul. Adama Mickiewicza 36",52,21,765,Mickiewicza,Pułtusk,Pułtusk,pułtuski,MAZOWIECKIE
204,17179,"Rzeszów, ul. Starzyńskiego",50,21,810,Starzyńskiego 17,Rzeszów,Rzeszów,Rzeszów,PODKARPACKIE
205,17182,"Kętrzyn, ul. Wojska Polskiego",54,21,357,Wojska Polskiego 9,Kętrzyn,Kętrzyn,kętrzyński,WARMIŃSKO-MAZURSKIE
206,17438,"Kościan, ul. Maya",52,16,404,Maya,Kościan,Kościan,kościański,WIELKOPOLSKIE


In [39]:
df2 = df[['gegrLat','gegrLon']]
list_of_loc = df2.to_dict('records')
loc = []

for item in list_of_loc:
    one_loc = item['gegrLat'], item['gegrLon']
    loc.append(one_loc)

print(loc)


[(50, 15), (50, 16), (50, 16), (51, 16), (50, 17), (51, 15), (50, 15), (50, 16), (51, 17), (51, 17), (51, 17), (51, 15), (53, 18), (52, 18), (53, 18), (53, 17), (53, 17), (52, 18), (53, 18), (53, 18), (53, 18), (52, 19), (53, 17), (52, 23), (51, 22), (51, 22), (51, 22), (50, 23), (52, 19), (51, 19), (51, 19), (51, 19), (51, 19), (51, 19), (51, 19), (51, 19), (52, 15), (52, 15), (52, 15), (51, 16), (51, 15), (51, 15), (50, 19), (50, 19), (50, 20), (49, 20), (49, 19), (49, 21), (50, 21), (50, 19), (50, 20), (49, 19), (51, 20), (52, 20), (52, 20), (52, 21), (52, 20), (52, 19), (52, 19), (51, 21), (52, 22), (52, 21), (52, 20), (52, 21), (52, 21), (52, 20), (50, 18), (50, 18), (50, 17), (50, 18), (53, 23), (53, 23), (53, 23), (53, 22), (50, 22), (49, 21), (49, 21), (50, 22), (49, 22), (50, 22), (49, 22), (50, 21), (54, 17), (54, 18), (54, 18), (54, 18), (54, 17), (54, 19), (54, 17), (54, 18), (54, 17), (50, 20), (50, 21), (51, 21), (49, 19), (50, 19), (50, 19), (50, 19), (50, 18), (50, 19),

In [40]:
import folium

gegrLat = 49.293564
gegrLon = 19.960083

location = gegrLat, gegrLon

m = folium.Map(location=location, width=800, height=400)
folium.Marker(location).add_to(m)
m

In [28]:
from geopy.geocoders import Nominatim
 
#address we need to geocode
loc = 'Bydgoszcz wyspa młyńska'
 
#making an instance of Nominatim class
geolocator = Nominatim(user_agent="my_request")
 
#applying geocode method to get the location
location2 = geolocator.geocode(loc)
 
#printing address and coordinates
print(location2.address)
print((location2.latitude, location2.longitude))


Wyspa Młyńska, Stare Miasto, Śródmieście-Bocianowo-Stare Miasto, Bydgoszcz, województwo kujawsko-pomorskie, 85-112, Polska
(53.122624349999995, 17.995532823870633)


In [23]:
from geopy.distance import distance

gegrLat = 49.293564
gegrLon = 19.960083

location = gegrLat, gegrLon

gegrLat2 = 53.161680
gegrLon2 = 17.9078569

location5 = gegrLat2, gegrLon2

km = distance(location, location5)
km

Distance(453.53703823898144)